## Enriching stock market data using Open AI API 

<p align="center">
    <img src="images/nasdaq100.png" width="450">
</p>

The Nasdaq-100 is a stock market index made up of 101 equity securities issued by 100 of the largest non-financial companies listed on the Nasdaq stock exchange. It helps investors compare stock prices with previous prices to determine market performance.

In this project you are provided with two CSV files containing Nasdaq-100 stock information:
- _**nasdaq100_CA.csv**_: contains information about companies in the index such as symbol, name, etc. For this analysis, only companies headquartered in California have been selected.
- _**nasdaq100_price_change.csv**_: contains price changes per stock across periods including (but not limited to) one day, five days, one month, six months, one year, etc.

As an AI developer, you will leverage the OpenAI API to classify companies into sectors and produce a summary of sector and company performance for this year, for the companies in the index that are headquartered in California.

# CSV with Nasdaq-100 stock data

In this project, you have available two CSV files `nasdaq100_CA.csv` and `nasdaq100_price_change.csv`.

## nasdaq100_CA.csv

```py
symbol,name,headQuarter,dateFirstAdded,cik,founded
AAPL,Apple Inc.,"Cupertino, CA",,0000320193,1976-04-01
ABNB,Airbnb,"San Francisco, CA",,0001559720,2008-08-01
ADBE,Adobe Inc.,"San Jose, CA",,0000796343,1982-12-01
...
```

## nasdaq100_price_change.csv

```py
symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
AAPL,-1.7254,-8.30086,-6.20411,3.042,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
ADBE,0.5409,-1.77817,9.16191,52.0465,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.09735,63.42847,92.81874,286.77518,26012.63736
...
```

In [9]:
# Start your code here!
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL = "gpt-3.5-turbo"

nasdaq100_ca = pd.read_csv("nasdaq100_CA.csv")
price_change = pd.read_csv("nasdaq100_price_change.csv")

nasdaq100_ca = nasdaq100_ca.merge(price_change[["symbol", "ytd"]], on="symbol", how="left")
nasdaq100_ca["ytd"] = pd.to_numeric(nasdaq100_ca["ytd"], errors="coerce")

sectors = (
    "Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, "
    "Communication, Energy, Consumer Defensive, Real Estate, Financial"
)

def classify_sector(company_name):
    prompt = (
        f'Classify the company "{company_name}" into ONE sector from: {sectors}. '
        f"Return ONLY the sector name."
    )
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    return resp.choices[0].message.content.strip()

nasdaq100_ca["sector"] = nasdaq100_ca["name"].apply(classify_sector)

nasdaq = nasdaq100_ca.copy()

recommend_prompt = (
    "Provide summary information about Nasdaq-100 companies headquartered in CA, "
    "recommending the two best sectors and two or more companies per sector, "
    "based on year-to-date (YTD) performance.\n\n"
    f"Company data:\n{nasdaq[['symbol','name','sector','ytd']].to_string(index=False)}"
)

response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": recommend_prompt}],
    temperature=0.0,
)

stock_recommendations = response.choices[0].message.content
print(stock_recommendations)


Based on year-to-date performance, the two best sectors among Nasdaq-100 companies headquartered in California are Technology and Healthcare.

In the Technology sector, the top-performing companies are:
1. NVDA (Nvidia) with a YTD performance of 217.26511%
2. META (Meta Platforms) with a YTD performance of 153.77585%
3. AMD (Advanced Micro Devices Inc.) with a YTD performance of 82.45861%
4. ADBE (Adobe Inc.) with a YTD performance of 57.22723%

In the Healthcare sector, the top-performing companies are:
1. DXCM (DexCom) with a YTD performance of 5.43886%
2. ISRG (Intuitive Surgical) with a YTD performance of 16.41757%
